# EnergyPlus and TRNSYS Example

In [1]:
import os
from zerobnl import CoSim

You can safely ignore the following error (it will also be in the nodes logs):

RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88

-> [Numpy documentation](https://github.com/numpy/numpy/pull/432)

In [2]:
sim = CoSim()

# There are 4 environments in this example.

sim.create_meta_model("MetaPLANT", [("COG_QDH", "kW"),("COG_FlagIN","-"),("COG_mdotTOT", "kgs")], [("COG_Tout", "Cdeg"),("COG_MDOTtoTES", "kgs"),("COG_MDOTfromTES", "kgs"),("COG_TTESin", "Cdeg"),("COG_soo", "-"),("COG_MDOTtoLOAD", "kgs")]) # Set, Get
sim.create_environment("EnvPLANT", "wrapper_plant.py", "Dockerfile_na")

sim.create_meta_model("MetaLSHP", [("LSHP_QDH", "kW"),("LSHP_FlagIN", "-")], [("LSHP_ToutL", "Cdeg"),("LSHP_MDOTtot", "kgs"),("LSHP_MDOTtoTES", "kgs"),("LSHP_MDOTfromTES", "kgs"),("LSHP_TTESin", "Cdeg"),("LSHP_soo", "-"),("LSHP_MDOTtoLOAD","kgs")]) # Set, Get
sim.create_environment("EnvLSHP", "wrapper_lshp.py", "Dockerfile_lshp")

sim.create_meta_model("MetaTES", [("TES_FlagIN", "-"),("TES_Tin","Cdeg"),("TES_MDOTinC","kgs"),("TES_MDOTinD","kgs")],[("TES_ToutD", "Cdeg"),("TES_socOUT", "-"),("TES_MDOToutD", "kgs"),("TES_soo", "-")]) # Set, Get
sim.create_environment("EnvTES", "wrapper_tes.py", "Dockerfile_tes")

sim.create_meta_model("MetaHOB", [("HOB_Tset", "Cdeg"),("HOB_FlagIN","-"),("HOB_mdotIN","kgs"),("HOB_Tin","Cdeg")],[]) # Set, Get
sim.create_environment("EnvHOB", "wrapper_hob.py", "Dockerfile_hob")

sim.create_meta_model("MetaDEM1", [("Tthermostat", "Cdeg")], [("QDH", "W"),("Tindoor","Cdeg")]) # Set, Get
sim.create_environment("EnvDEM1", "wrapper_dem1.py", "Dockerfile_na")
sim.create_meta_model("MetaDEM2", [("Tthermostat", "Cdeg")], [("QDH", "W"),("Tindoor","Cdeg")]) # Set, Get
sim.create_environment("EnvDEM2", "wrapper_dem2.py", "Dockerfile_na")

sim.create_meta_model("MetaCTRL", [("demandOK", "-"),("demandOK_mdot", "-"),("demandOK_price", "-")], [("COG_FlagOUT", "-"),("HOB_FlagOUT", "-"),("TES_FlagOUT", "-")]) # Set, Get
sim.create_environment("EnvCTRL", "wrapper_ctrl.py", "Dockerfile_ctrl")

sim.create_meta_model("MetaLCTRL", [("TindoorIN_0", "Cdeg"),("TindoorIN_1", "Cdeg"),("mdotTOT", "kgs"),("TES_socIN", "-"),("ToutdoorP", "Cdeg")], [("Tth_0", "Cdeg"),("Tth_1", "Cdeg"),("demandFlag", "-"),("demandFlag_mdot", "-"),("TsetP", "Cdeg")]) # Set, Get
sim.create_environment("EnvLCTRL", "wrapper_lctrl.py", "Dockerfile_lctrl")

sim.create_meta_model("MetaPCTRL", [], [("demandFlag_price", "-")]) # Set, Get
sim.create_environment("EnvPCTRL", "wrapper_pctrl.py", "Dockerfile_pctrl")

sim.create_meta_model("MetaWEATHER", [], [("TdryBulb", "Cdeg")]) # Set, Get
sim.create_environment("EnvWEATHER", "wrapper_weather.py", "Dockerfile_weather")

sim.create_meta_model("MetaHNET", [("HNET_QDH_1", "W"),("HNET_QDH_2", "W"),("HNET_TsSET", "Cdeg")], [("HNET_QDH", "kW"),("HNET_MDOTtot", "kgs")]) # Set, Get
sim.create_environment("EnvHNET", "wrapper_shnet.py", "Dockerfile_shnet")

sim.create_meta_model("MetaISEL", [("soo1", "-"),("mdot1", "kgs"),("Tin1", "Cdeg"),("soo2", "-"),("mdot2", "kgs"),("Tin2", "Cdeg")], [("mdot", "kgs"),("Tout", "Cdeg")]) # Set, Get
sim.create_environment("EnvISEL", "wrapper_isel.py", "Dockerfile_isel")

# There is one model per each environment in this example. 
# OBS!! The name Base# indicates a model. Each model is contained in a node. There can be more nodes in each environment.
sim.add_node("Base0", "MetaLCTRL", "EnvLCTRL", init_values={},files=['TS_set.txt'],local=False)
sim.add_node("Base0_1", "MetaPCTRL", "EnvPCTRL", init_values={},files=['COGWaste_cost.txt','LSHP_cost.txt'],local=False)
sim.add_node("Base1", "MetaCTRL", "EnvCTRL", init_values={},local=False)
sim.add_node("Base2", "MetaPLANT", "EnvPLANT", init_values={"COG_QDH":2000,"COG_FlagIN":1}, files=["COGplant.fmu"],local=True)
#sim.add_node("Base2_1", "MetaLSHP", "EnvLSHP", init_values={},local=False)
sim.add_node("Base3", "MetaDEM1", "EnvDEM1", init_values={}, files=["Residential_DH_1.fmu"],local=True)
sim.add_node("Base3_1", "MetaDEM2", "EnvDEM2", init_values={}, files=["Residential_DH_2.fmu"],local=True)
sim.add_node("Base4", "MetaHOB", "EnvHOB", init_values={},local=False)
sim.add_node("Base5", "MetaTES", "EnvTES", init_values={},local=False)
#sim.add_node("Base5_1", "MetaTES", "EnvTES", init_values={},local=False)
sim.add_node("Base6", "MetaWEATHER", "EnvWEATHER", init_values={},files=['SWE_Stockholm.Arlanda.024600_IWEC.epw'],local=False)
sim.add_node("Base7", "MetaHNET", "EnvHNET", init_values={},local=False)
sim.add_node("Base8", "MetaISEL", "EnvISEL", init_values={},local=False)

# Links among nodes
sim.add_link("Base0", "Tth_0", "Base3", "Tthermostat") # ok - Control on demand (Thermostat set point)
sim.add_link("Base0", "Tth_1", "Base3_1", "Tthermostat") # ok - Control on demand (Thermostat set point)
sim.add_link("Base0", "demandFlag", "Base1", "demandOK") # ok - Precontrol on the supply units based on weather (Tset) and demand (mdot) and availability of capacity in the buildings (Tindoor)
sim.add_link("Base0", "demandFlag_mdot", "Base1", "demandOK_mdot") # ok - Precontrol on the supply units based on weather (Tset) and demand (mdot) and availability of capacity in the buildings (Tindoor) 
sim.add_link("Base0_1", "demandFlag_price", "Base1", "demandOK_price")
sim.add_link("Base0", "TsetP", "Base4", "HOB_Tset") # ok - Information passed onto the HOB to full fill the load's request
sim.add_link("Base0", "TsetP", "Base7", "HNET_TsSET") # ok - Information passed onto the HOB to full fill the load's request

sim.add_link("Base1", "COG_FlagOUT", "Base2", "COG_FlagIN") # ok - Control on COG 
sim.add_link("Base1", "HOB_FlagOUT", "Base4", "HOB_FlagIN") # ok - Control on HOB
sim.add_link("Base1", "TES_FlagOUT", "Base5", "TES_FlagIN") # ok - Control on TES

sim.add_link("Base2", "COG_MDOTtoTES", "Base5", "TES_MDOTinC") # ok Total - Direct interaction COG-TES
sim.add_link("Base2", "COG_MDOTfromTES", "Base5", "TES_MDOTinD") # ok Total - Direct interaction COG-TES
sim.add_link("Base2", "COG_TTESin", "Base5", "TES_Tin") # ok - Direct interaction COG-TES
sim.add_link("Base2", "COG_Tout", "Base8", "Tin1") # ok Total - Direct interaction COG-TES
sim.add_link("Base2", "COG_MDOTtoLOAD", "Base8", "mdot1") # ok - Direct interaction COG-TES
sim.add_link("Base2", "COG_soo", "Base8", "soo1") # ok - Direct interaction COG-TES

sim.add_link("Base3", "QDH", "Base7", "HNET_QDH_1") # ok - Feedback to the supply system
sim.add_link("Base3", "Tindoor", "Base0", "TindoorIN_0") # ok - Feedback to the local control to define the supply units (1)
sim.add_link("Base3_1", "Tindoor", "Base0", "TindoorIN_1") # ok - Feedback to the local control to define the supply units (1)
sim.add_link("Base3_1", "QDH", "Base7", "HNET_QDH_2") # ok - Feedback to the supply system

sim.add_link("Base5", "TES_ToutD", "Base8", "Tin2") # ok - Direct interaction TES - HOB (dis)
sim.add_link("Base5", "TES_MDOToutD", "Base8", "mdot2") # ok Total - Direct interaction TES - HOB (dis)
sim.add_link("Base5", "TES_soo", "Base8", "soo2") # ok Total - Direct interaction TES - HOB (dis)
sim.add_link("Base5", "TES_socOUT", "Base0", "TES_socIN") # ok

sim.add_link("Base6", "TdryBulb", "Base0", "ToutdoorP") # ok

sim.add_link("Base7", "HNET_QDH", "Base2", "COG_QDH") # ok
sim.add_link("Base7", "HNET_MDOTtot", "Base2", "COG_mdotTOT") # ok Total
sim.add_link("Base7", "HNET_MDOTtot", "Base0", "mdotTOT") # ok Total

sim.add_link("Base8", "mdot", "Base4", "HOB_mdotIN") # ok Total - Direct interaction COG-HOB
sim.add_link("Base8", "Tout", "Base4", "HOB_Tin") # ok - Direct interaction COG-HOB

# Create groups from the simulation sequence. Nodes in the same group run in parallel. 
# A group is defined within the first level of square brackets.
sim.create_sequence([["Base3","Base3_1"],["Base7"],["Base2"],["Base5"],["Base8"],["Base4"],["Base6"],["Base0","Base0_1"],["Base1"]])
sim.set_time_unit("seconds")
sim.create_steps([3600] * 24)

Once the next step has been launched, logging `INFO :: Waiting for local nodes to run..`, you need to run tho following command `wrapper_eplus.py Base1 GRP1` in the indicated folder (in a dedicated environment) in order to run the local node.

In [3]:
sim.run()

INFO :: Waiting for local nodes to run...
INFO :: Simulation finished in 0 min and 35 sec


If you see `INFO :: Simulation finished in X min and Y sec` it means everything went well.
You can find logs of the nodes in the file `nodes.log`, it's a text file you can open it directly in Jupyter or in your favorite text editor.

At the begining of the file you will find a serie of:

`Step X/10 : DO SOMETHING
 ---> 29d2f3226daf`
 
It's the logs of the creation of the Docker image, based on the provided Dockerfile (here `Dockerfile_base`).

Then all the logs are structures in the same way:

`<node>    | <level> :: <message>`

* `node` refers to the concerned simulation node or orchestrator
* `level` can be `DEBUG`: used for development purpose, `INFO`: giving you info on the running process, `WARNING`: warning you on action to make or some weird behaviour, `ERROR`: something went wrong and `CRITICAL`: something went really wrong.
* `message` is the body of the log, it describes what's happening.

You can also find information on the ongoing simulation in the file `activity.log` (in the root folder for the main processus and on the temporary folder for each node)

In [4]:
sim.connect_to_results_db()
sim.get_list_of_available_results()

,IN/OUT,Node,Attribute
0,IN,Base3,Tthermostat
1,IN,Base8,Tin2
2,IN,Base4,HOB_mdotIN
3,OUT,Base3,QDH
4,OUT,Base1,COG_FlagOUT
5,OUT,Base5,TES_socOUT
6,OUT,Base8,mdot
7,IN,Base8,mdot2
8,IN,Base5,TES_MDOTinC
9,OUT,Base0_1,demandFlag_price


The name to the stored results are build as `<type>||<node>||<attribute>`.

`type` can be:
* `IN` if it's an input attribute (to set - stored automatically)
* `OUT` if it's an output attribute (to get - stored automatically)
* `X` if it's an internal value (stored by the user, using the `save_attribute()` method in the wrapper)

Knowing this, you can create matching pattern using `*` in order to properly select results.

In [5]:
file1 = open('Results_Tset_Th_demFlag.csv','w')
for key, value in sim.get_results_by_pattern("OUT*Base0*").items():
    print(key)
    print(value) 
    file1.write(str(value))
file1.close()

INFO :: Matching results: Base0_1 - demandFlag_price
INFO :: Matching results: Base0 - demandFlag_mdot
INFO :: Matching results: Base0 - TsetP
INFO :: Matching results: Base0 - demandFlag
INFO :: Matching results: Base0 - Tth_0
INFO :: Matching results: Base0 - Tth_1


OUT||Base0_1||demandFlag_price
2000-01-01 01:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 09:00:00    1.0
2000-01-01 10:00:00    1.0
2000-01-01 11:00:00    1.0
2000-01-01 12:00:00    1.0
2000-01-01 13:00:00    1.0
2000-01-01 14:00:00    1.0
2000-01-01 15:00:00    1.0
2000-01-01 16:00:00    1.0
2000-01-01 17:00:00    1.0
2000-01-01 18:00:00    1.0
2000-01-01 19:00:00    1.0
2000-01-01 20:00:00    1.0
2000-01-01 21:00:00    1.0
2000-01-01 22:00:00    1.0
2000-01-01 23:00:00    1.0
2000-01-02 00:00:00    1.0
dtype: float64
OUT||Base0||TsetP
2000-01-01 01:00:00    100.00
2000-01-01 02:00:00    100.00
2000-01-01 03:00:00    100.00
2000-01-01 04:00:00    100.00
2000-01-01 05:00:00    100.00
2000-01-01 06:00:00    100.00
2000-01-01 07:00:00    100.00
2000-01-01 08:00:00    100.00
2000-01-01 09:00:00    100.00
2000-01-01 10:00:0

In [6]:
file2 = open('Results_Flags.csv','w')
for key, value in sim.get_results_by_pattern("OUT*Base1*").items():
    print(key)
    print(value) 
    file2.write(str(value))
file2.close()

INFO :: Matching results: Base1 - COG_FlagOUT
INFO :: Matching results: Base1 - TES_FlagOUT
INFO :: Matching results: Base1 - HOB_FlagOUT


OUT||Base1||COG_FlagOUT
2000-01-01 01:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 09:00:00    1.0
2000-01-01 10:00:00    1.0
2000-01-01 11:00:00    1.0
2000-01-01 12:00:00    1.0
2000-01-01 13:00:00    1.0
2000-01-01 14:00:00    1.0
2000-01-01 15:00:00    1.0
2000-01-01 16:00:00    1.0
2000-01-01 17:00:00    1.0
2000-01-01 18:00:00    1.0
2000-01-01 19:00:00    1.0
2000-01-01 20:00:00    1.0
2000-01-01 21:00:00    1.0
2000-01-01 22:00:00    1.0
2000-01-01 23:00:00    1.0
2000-01-02 00:00:00    1.0
dtype: float64
OUT||Base1||HOB_FlagOUT
2000-01-01 01:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 06:00:00   -1.0
2000-01-01 07:00:00   -1.0
2000-01-01 08:00:00   -1.0
2000-01-01 09:00:00    1.0
2000-01-01 10:00:00    1.0
2000-01-01 11:00:00

In [7]:
file3 = open('Results_COGout.csv','w')
for key, value in sim.get_results_by_pattern("OUT*Base2*").items():
    print(key)
    print(value) 
    file3.write(str(value))
file3.close()

INFO :: Matching results: Base2 - COG_soo
INFO :: Matching results: Base2 - COG_TTESin
INFO :: Matching results: Base2 - COG_MDOTtoLOAD
INFO :: Matching results: Base2 - COG_MDOTfromTES
INFO :: Matching results: Base2 - COG_Tout
INFO :: Matching results: Base2 - COG_MDOTtoTES


OUT||Base2||COG_MDOTfromTES
2000-01-01 01:00:00      0.000000
2000-01-01 02:00:00      0.000000
2000-01-01 03:00:00      0.000000
2000-01-01 04:00:00      0.000000
2000-01-01 05:00:00      0.000000
2000-01-01 06:00:00    212.386665
2000-01-01 07:00:00    202.615351
2000-01-01 08:00:00    221.598130
2000-01-01 09:00:00    202.276386
2000-01-01 10:00:00    212.736605
2000-01-01 11:00:00    226.738348
2000-01-01 12:00:00    192.293211
2000-01-01 13:00:00    196.703831
2000-01-01 14:00:00    206.938100
2000-01-01 15:00:00    206.194539
2000-01-01 16:00:00    209.408173
2000-01-01 17:00:00    184.897601
2000-01-01 18:00:00    183.684499
2000-01-01 19:00:00     23.634587
2000-01-01 20:00:00    178.126525
2000-01-01 21:00:00      0.000000
2000-01-01 22:00:00      0.000000
2000-01-01 23:00:00      0.000000
2000-01-02 00:00:00      0.000000
dtype: float64
OUT||Base2||COG_MDOTtoLOAD
2000-01-01 01:00:00      0.0
2000-01-01 02:00:00      0.0
2000-01-01 03:00:00      0.0
2000-01-01 04:00:00      0.

In [8]:
file4 = open('Results_Tindoor_QR.csv','w')
for key, value in sim.get_results_by_pattern("OUT*Base3*").items():
    print(key)
    print(value) 
    file4.write(str(value))
file4.close()

INFO :: Matching results: Base3 - QDH
INFO :: Matching results: Base3_1 - QDH
INFO :: Matching results: Base3_1 - Tindoor
INFO :: Matching results: Base3 - Tindoor


OUT||Base3_1||QDH
2000-01-01 01:00:00    7.279300e-316
2000-01-01 02:00:00    7.279281e-316
2000-01-01 03:00:00    7.279285e-316
2000-01-01 04:00:00    7.279281e-316
2000-01-01 05:00:00    7.279281e-316
2000-01-01 06:00:00     2.000000e+03
2000-01-01 07:00:00     2.000000e+03
2000-01-01 08:00:00     1.990526e+03
2000-01-01 09:00:00     1.979927e+03
2000-01-01 10:00:00     2.000000e+03
2000-01-01 11:00:00     2.000000e+03
2000-01-01 12:00:00     2.000000e+03
2000-01-01 13:00:00     2.000000e+03
2000-01-01 14:00:00     2.000000e+03
2000-01-01 15:00:00     2.000000e+03
2000-01-01 16:00:00     2.000000e+03
2000-01-01 17:00:00     2.000000e+03
2000-01-01 18:00:00     2.000000e+03
2000-01-01 19:00:00     1.474614e+03
2000-01-01 20:00:00     1.862247e+03
2000-01-01 21:00:00    7.279284e-316
2000-01-01 22:00:00    7.279290e-316
2000-01-01 23:00:00    7.279290e-316
2000-01-02 00:00:00    7.279290e-316
dtype: float64
OUT||Base3_1||Tindoor
2000-01-01 01:00:00    16.520383
2000-01-01 02:00:00    1

In [9]:
file5 = open('Results_TESout.csv','w')
for key, value in sim.get_results_by_pattern("OUT*Base5*").items():
    print(key)
    print(value) 
    file5.write(str(value))
file5.close()

INFO :: Matching results: Base5 - TES_socOUT
INFO :: Matching results: Base5 - TES_soo
INFO :: Matching results: Base5 - TES_MDOToutD
INFO :: Matching results: Base5 - TES_ToutD


OUT||Base5||TES_MDOToutD
2000-01-01 01:00:00    507.0
2000-01-01 02:00:00    507.0
2000-01-01 03:00:00    507.0
2000-01-01 04:00:00    507.0
2000-01-01 05:00:00    507.0
2000-01-01 06:00:00    507.0
2000-01-01 07:00:00    507.0
2000-01-01 08:00:00    507.0
2000-01-01 09:00:00    507.0
2000-01-01 10:00:00    507.0
2000-01-01 11:00:00    507.0
2000-01-01 12:00:00    507.0
2000-01-01 13:00:00    507.0
2000-01-01 14:00:00    507.0
2000-01-01 15:00:00    507.0
2000-01-01 16:00:00    507.0
2000-01-01 17:00:00    507.0
2000-01-01 18:00:00    507.0
2000-01-01 19:00:00    507.0
2000-01-01 20:00:00    507.0
2000-01-01 21:00:00    507.0
2000-01-01 22:00:00    507.0
2000-01-01 23:00:00    507.0
2000-01-02 00:00:00    507.0
dtype: float64
OUT||Base5||TES_ToutD
2000-01-01 01:00:00    75.0
2000-01-01 02:00:00    75.0
2000-01-01 03:00:00    75.0
2000-01-01 04:00:00    75.0
2000-01-01 05:00:00    75.0
2000-01-01 06:00:00    75.0
2000-01-01 07:00:00    75.0
2000-01-01 08:00:00    75.0
2000-01-01 09:00:0

In [10]:
file6 = open('Results_Toutdoor.csv','w')
for key, value in sim.get_results_by_pattern("OUT*Base6*").items():
    print(key)
    print(value) 
    file6.write(str(value))
file6.close()

INFO :: Matching results: Base6 - TdryBulb


OUT||Base6||TdryBulb
2000-01-01 01:00:00   -17.0
2000-01-01 02:00:00   -17.0
2000-01-01 03:00:00   -17.0
2000-01-01 04:00:00   -17.0
2000-01-01 05:00:00   -16.9
2000-01-01 06:00:00   -16.7
2000-01-01 07:00:00   -16.6
2000-01-01 08:00:00   -15.8
2000-01-01 09:00:00   -15.0
2000-01-01 10:00:00   -14.2
2000-01-01 11:00:00   -13.1
2000-01-01 12:00:00   -12.1
2000-01-01 13:00:00   -11.0
2000-01-01 14:00:00   -10.7
2000-01-01 15:00:00   -10.5
2000-01-01 16:00:00   -10.2
2000-01-01 17:00:00    -9.8
2000-01-01 18:00:00    -9.4
2000-01-01 19:00:00    -9.0
2000-01-01 20:00:00    -8.0
2000-01-01 21:00:00    -7.1
2000-01-01 22:00:00    -6.1
2000-01-01 23:00:00    -5.7
2000-01-02 00:00:00    -5.4
dtype: float64


In [11]:
for key, value in sim.get_results_by_pattern("OUT*Base7*").items():
    print(key)
    print(value) 

INFO :: Matching results: Base7 - HNET_MDOTtot
INFO :: Matching results: Base7 - HNET_QDH


OUT||Base7||HNET_MDOTtot
2000-01-01 01:00:00      0.000000
2000-01-01 02:00:00      0.000000
2000-01-01 03:00:00      0.000000
2000-01-01 04:00:00      0.000000
2000-01-01 05:00:00      0.000000
2000-01-01 06:00:00    379.249085
2000-01-01 07:00:00    374.098474
2000-01-01 08:00:00    384.105774
2000-01-01 09:00:00    373.919671
2000-01-01 10:00:00    379.434134
2000-01-01 11:00:00    386.815640
2000-01-01 12:00:00    368.656690
2000-01-01 13:00:00    370.981901
2000-01-01 14:00:00    376.377252
2000-01-01 15:00:00    375.985258
2000-01-01 16:00:00    377.679436
2000-01-01 17:00:00    364.757837
2000-01-01 18:00:00    367.026085
2000-01-01 19:00:00    286.553124
2000-01-01 20:00:00    376.086003
2000-01-01 21:00:00      0.000000
2000-01-01 22:00:00      0.000000
2000-01-01 23:00:00      0.000000
2000-01-02 00:00:00      0.000000
dtype: float64
OUT||Base7||HNET_QDH
2000-01-01 01:00:00        0.000000
2000-01-01 02:00:00        0.000000
2000-01-01 03:00:00        0.000000
2000-01-01 04:0

In [12]:
for key, value in sim.get_results_by_pattern("OUT*Base0_1*").items():
    print(key)
    print(value) 

INFO :: Matching results: Base0_1 - demandFlag_price


OUT||Base0_1||demandFlag_price
2000-01-01 01:00:00    1.0
2000-01-01 02:00:00    1.0
2000-01-01 03:00:00    1.0
2000-01-01 04:00:00    1.0
2000-01-01 05:00:00    1.0
2000-01-01 06:00:00    1.0
2000-01-01 07:00:00    1.0
2000-01-01 08:00:00    1.0
2000-01-01 09:00:00    1.0
2000-01-01 10:00:00    1.0
2000-01-01 11:00:00    1.0
2000-01-01 12:00:00    1.0
2000-01-01 13:00:00    1.0
2000-01-01 14:00:00    1.0
2000-01-01 15:00:00    1.0
2000-01-01 16:00:00    1.0
2000-01-01 17:00:00    1.0
2000-01-01 18:00:00    1.0
2000-01-01 19:00:00    1.0
2000-01-01 20:00:00    1.0
2000-01-01 21:00:00    1.0
2000-01-01 22:00:00    1.0
2000-01-01 23:00:00    1.0
2000-01-02 00:00:00    1.0
dtype: float64
